In [ ]:
from ultralytics import YOLO

In [ ]:
# Cargar modelo
model = YOLO('yolov8n.yaml') # Construir nuevo modelo desde yaml

results = model.train(data='license_plates_dataset/data.yaml', epochs=100, imgsz=640, patience=20)
